In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle

with open('/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/dict_youtube_mapping.pkl', 'rb') as f:
    data = pickle.load(f)
print(len(data))

1970


In [ ]:
!mkdir local_features1

In [ ]:
!mkdir motion_features1

***Unzipping the PRE-EXTRACTED features of MSVD dataset from the fc7 LAYER OF FASTER R-CNN NETWORK and the number is 8***

In [ ]:
!unzip '/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/rcnn8_msvd.zip'

In [ ]:
import shutil

directory_path = '/content/motion_features1'

# Delete all files in the directory
shutil.rmtree(directory_path)


***Process the HDF5 files and saves them as .npy files in a ZIP format, later used after unzipping***

In [ ]:
import h5py
import numpy as np

def extract_local_features(save=False):
  for i in range(1970):
    idx = i+1
    filename = '/content/home/sdc/tuyunbin/msvd/rcnn8/msvd_vid'+str(idx)+'_rcnn.h5'
    vid = list(data.keys())[list(data.values()).index('vid'+str(idx))]
    with h5py.File(filename, "r") as f:
      a_group_key = list(f.keys())[0]
      feature_vector = list(f[a_group_key])
      feature_vector = np.array(feature_vector)
      feature_vector = np.mean(feature_vector,axis=1)
      if feature_vector.shape[0] < 28:
        while feature_vector.shape[0] != 28:
          feature_vector = np.concatenate((feature_vector,[feature_vector[-1]]))
    feature_path='local_features1/'+vid+'.npy'
    np.save(feature_path, feature_vector)

    print(i,':',feature_vector.shape,',',vid)


  if save is True:
    !zip -r local_features1.zip local_features1
    !cp 'local_features1.zip' '/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/'

***Importing the necessary libraries***

In [4]:
import os
import cv2
import random
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import functools, operator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.inception_v3 import InceptionV3

***Reading the .csv file from the original MSVD dataset***

In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/data.csv')
df

,VideoID,Description
0,-4wsuPCjDBc_5_15,a squirrel is eating a peanut in it s shell
1,-4wsuPCjDBc_5_15,a chipmunk is eating
2,-4wsuPCjDBc_5_15,a chipmunk is eating a peanut
3,-4wsuPCjDBc_5_15,a chipmunk is eating a nut
4,-4wsuPCjDBc_5_15,a squirrel is eating a nut
...,...,...
80783,zxB4dFJhHR8_1_9,a girl riding a bicycle
80784,zxB4dFJhHR8_1_9,a smiling girl wearing backpack is riding a bike
80785,zxB4dFJhHR8_1_9,the girl rode her bike at the beach
80786,zzit5b_-ukg_5_20,a boy is doing exercise by cycle


In [ ]:
videoIDs = list(df.VideoID.unique())
print(len(videoIDs))

1970


***GLOBAL FEATURE EXTRACTION***

In [ ]:
def extract_frames_equally_spaced(frames, K):
        n_frames = len(frames)
        splits = np.array_split(range(n_frames), K)
        idx_taken = [s[0] for s in splits]
        sub_frames = []

        for idx in idx_taken:
          sub_frames.append(frames[idx])
        return sub_frames

def pad_frames(frames, limit, jpegs=False):
        last_frame = frames[-1]
        if jpegs:
            frames_padded = frames + [last_frame]*(limit-len(frames))
        else:
            padding = np.asarray([last_frame * 0.]*(limit-len(frames)))
            frames_padded = np.concatenate([frames, padding], axis=0)
        return frames_padded

def video_to_frames(input_loc,K,motion=False):
    cap = cv2.VideoCapture(input_loc)

    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    count = 0

    if motion is True:
      K = max(video_length,50)

    frames=[]
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
        frames.append(frame)
        count = count + 1

        if (count > (video_length-1)):
            cap.release()
            break
    if len(frames) < K:
        frames = pad_frames(frames,K,True)
    else:
        frames = extract_frames_equally_spaced(frames,K)

    return frames

def resize_frames(frames):
  new_frames = []
  for frame in frames:
    new_frame = cv2.resize(frame,(224,224))
    new_frames.append(new_frame)

  return new_frames

K=28

In [ ]:
#Googlenet model
def get_model():
  i = tf.keras.layers.Input([224, 224, 3], dtype = tf.float32)
  x = tf.keras.applications.inception_v3.preprocess_input(i)
  model = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3),pooling='avg')
  out = model(x)
  model=tf.keras.models.Model(inputs=[i], outputs=out)
  return model


In [5]:
import os

def count_files(directory):
    file_count = 0
    for _, _, files in os.walk(directory):
        file_count += len(files)
    return file_count

# Example usage
directory_path = '/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/global_features'
num_files = count_files(directory_path)
print(f"Number of files in '{directory_path}': {num_files}")


Number of files in '/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/global_features': 1970


***SHAPE OF THE GLOBAL FEATURES***

In [ ]:
import numpy as np

# Load the .npy file
file_path = '/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/global_features/-4wsuPCjDBc_5_15.npy'
data = np.load(file_path)

# Get the shape of the loaded data
shape = data.shape

print(f"Shape of the .npy file: {shape}")


Shape of the .npy file: (28, 2048)


In [ ]:
!pip install sk-video
!wget -O 'c3d.py' 'https://drive.google.com/uc?id=1RC9trsgIo2OsM8dV7CPg73drqSRZMr8j&confirm=t'
!wget -O 'sports1M_utils.py' 'https://drive.google.com/uc?id=1S49OSW2pCUPPD9F-763mG-aBx2gBOr2i&confirm=t'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.9 MB/s eta 0:00:00
--2023-06-21 09:17:28--  https://drive.google.com/uc?id=1RC9trsgIo2OsM8dV7CPg73drqSRZMr8j&confirm=t
Resolving drive.google.com (drive.google.com)... 74.125.141.113, 74.125.141.100, 74.125.141.138, ...
Connecting to drive.google.com (drive.google.com)|74.125.141.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-20-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dbjo3h1maltsj10pcnbhkp32g4qij0bp/1687338975000/17228267550831568869/*/1RC9trsgIo2OsM8dV7CPg73drqSRZMr8j?uuid=87d55034-2dba-48c7-9db3-c0d9c38307f0 [following]
--2023-06-21 09:17:28--  https://doc-0s-20-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/dbjo3h1maltsj10pcnbhkp32g4qij0bp/1687338975000/17228267550831568869/*/1RC9trsgIo2OsM8dV7CPg73drqSRZMr8j?uu

In [ ]:
!unzip '/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/local_features1.zip'

In [ ]:
import os

def count_files(directory):
    file_count = 0
    for _, _, files in os.walk(directory):
        file_count += len(files)
    return file_count

# Example usage
directory_path = '/content/local_features1'
num_files = count_files(directory_path)
print(f"Number of files in '{directory_path}': {num_files}")


Number of files in '/content/local_features1': 1970


***SHAPE OF THE LOCAL FEATURES***

In [ ]:
import numpy as np

# Load the .npy file
file_path = '/content/local_features1/-4wsuPCjDBc_5_15.npy'
data = np.load(file_path)

# Get the shape of the loaded data
shape = data.shape

print(f"Shape of the .npy file: {shape}")


Shape of the .npy file: (28, 4096)


In [ ]:
!mkdir motion_features1

***PROCESSING THE MOTION FEATURES***

In [ ]:
import h5py
import numpy as np

def extract_motion_features(save=False):
  for i in range(1970):
    idx = i+1
    filename = '/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/c3d_h5/msvd_vid'+str(idx)+'_c3d.h5'
    vid = list(data.keys())[list(data.values()).index('vid'+str(idx))]
    with h5py.File(filename, "r") as f:
      a_group_key = list(f.keys())[0]
      feature_vector = list(f[a_group_key])
      feature_vector = np.array(feature_vector)
      #feature_vector = np.mean(feature_vector,axis=1)
      if feature_vector.shape[0] < 28:
        while feature_vector.shape[0] != 28:
          feature_vector = np.concatenate((feature_vector,[feature_vector[-1]]))
          # print(feature_vector.shape,feature_vector)
      if feature_vector.shape[0] > 28:
        while feature_vector.shape[0] != 28:
          feature_vector = feature_vector[:28][:]
    feature_path='/content/motion_features1/'+vid+'.npy'
    np.save(feature_path, feature_vector)

    print(i,':',feature_vector.shape,',',vid)


  if save is True:
    !zip -r motion_features1.zip motion_features1
    !cp 'motion_features1.zip' '/content/drive/MyDrive/VIDEO CAPTIONING/video captioning/'

In [ ]:
extract_motion_features(save=True)